# Evaluating models

In this notebook i'm going to demonstrate various methods of evaluating models. Evaluating a model is a very important step of machine learning, i use it all the time to see if my engineered features, preprocessing steps and model parameters are resulting in a good model.

In [1]:
from sklearn.datasets import load_boston

X,y = load_boston(return_X_y=True)

## Evaluating: Splitting into train/test sets

First approach i'm going to show is splitting into a training and test set. We train the model using a training set and evaluate performance on a test set. This is the basic approach underlying most (all?) evaluation approaches.

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = LinearRegression()
model.fit(X_train, y_train)
print("Score: %0.2f" % model.score(X_test, y_test))

Score: 0.65


## Evaluating: Cross validated scoring

Sometimes, the above approach is a bit too simple. Score from training and evaluating on one possible split of data can be misleading. A more sophisticated approach can be to repeat above approach multiple times. Sklearn has the cross_val_score utility for this.

In the example below the data is split into a training and test set 5 times and a model is trained and scored repeatedly:

In [3]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X, y, cv=5)
print("Score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

Score: 0.35 (+/- 0.38)


While we intended the above to give a more reliable score, the opposite happened, score plummeted and standard deviation of scores is very large. Something is obviously wrong here!

Actually the data is not shuffled, possibly there is a trend over time. I'm not investigating this further here, however one way to combat this is shuffling data before splitting into training and test sets.

## Evaluating: Shuffled cross validated scoring

We can also specify how the data should be split during evaluation. In the above example we saw why that can be usefull!

Here i show how to use the KFold splitting utility, with shuffling:

In [4]:
from sklearn.model_selection import KFold

scores = cross_val_score(model, X, y, cv=KFold(n_splits=5, shuffle=True))
print("Score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

Score: 0.72 (+/- 0.06)


Another utility is ShuffleSplit, which allows to define test/train proportion independant of number of splits. This can be usefull because training set size can have an impact on quality of the model. With KFold splitting you end up with ever smaller test sets if you increase number of splits...

In [5]:
from sklearn.model_selection import ShuffleSplit

scores = cross_val_score(model, X, y, cv=ShuffleSplit(n_splits=25, test_size=0.2))
print("Score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

Score: 0.71 (+/- 0.05)


So i've demonstrated the most common evaluation approaches. Try them out and pick your favorite! I'm personally in favor of cross_val_score combined with ShuffleSplit, that gives me a decent feeling for model performance..